# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
city_data = pd.read_csv("city_dataframe.csv") 

city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Abu Samrah,35.30,37.18,297.54,47,0,5.55,SY,1597686414
1,1,Kapaa,22.08,-159.32,299.15,78,40,6.70,US,1597686414
2,2,Mar del Plata,-38.00,-57.56,287.15,30,0,9.80,AR,1597686415
3,3,Dorokhovo,55.55,36.37,292.15,59,5,3.00,RU,1597686415
4,4,Coquimbo,-29.95,-71.34,284.15,81,90,2.10,CL,1597686418


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [75]:
# We are assigning a variable inside a function
# gmaps.configure(api_key=g_key)

In [76]:
humidity_location = city_data[['Lat', 'Lng']].astype(float)
hum_intensity = city_data['Humidity'].astype(float)

In [103]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(humidity_location, weights=hum_intensity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
# A max temp < 80 degrees but > 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions.


# There are no places within my available cities that have == 0 cloudiness, so I'm using > 0 to test my data

hotel_df = city_data.loc[(city_data['Cloudiness'] > 0) & (city_data['Wind Speed'] > 10) & (((city_data['Max Temp']- 273.15) * 9/5 + 32) < 80) & (((city_data['Max Temp']- 273.15) * 9/5 + 32) > 70)]



hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,Ribeira Grande,38.52,-28.70,297.15,88,75,11.30,PT,1597686451
239,239,Bandarbeyla,9.49,50.81,297.66,81,15,15.05,SO,1597686521


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [203]:
hotel_df['Hotel Name'] = ""
hotel_df

C:\Users\asiah\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,Ribeira Grande,38.52,-28.70,297.15,88,75,11.30,PT,1597686451,
239,239,Bandarbeyla,9.49,50.81,297.66,81,15,15.05,SO,1597686521,


In [204]:
vaca_spot = "38.52, -28.70"
search_for = "Hotels"
within = 5000

parameters = {
    'location': vaca_spot,
    'keyword': search_for,
    'radius': within, 
    'key': g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Requests will autofill your parameters if they are in the form of a dictionary

hotels = requests.get(base_url, params = parameters)
hotel1 = hotels.json()['results'][0]['name']



vaca_spot = "9.49, 50.81"
search_for = "Hotels"
within = 5000

parameters = {
    'location': vaca_spot,
    'keyword': search_for,
    'radius': within, 
    'key': g_key
}
hotels = requests.get(base_url, params = parameters)
hotel2 = hotels.json()['results'][0]['name']

adding_hotels = [hotel1, hotel2]

# new_hotel_df = hotel_df.merge(adding_hotels, on='Hotel Name')
# new_hotel_df

hotel_df['Hotel Name'] = adding_hotels
hotel_df

C:\Users\asiah\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,Ribeira Grande,38.52,-28.70,297.15,88,75,11.30,PT,1597686451,Azul Singular
239,239,Bandarbeyla,9.49,50.81,297.66,81,15,15.05,SO,1597686521,Super secret pirate cove


In [205]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [206]:
# Add marker layer ontop of heat map

hotel_marker = hotel_df['Hotel Name'].tolist()

hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color = 'rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_marker)

# Display figure

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [207]:
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))